Example using dask-gateway with a JupyterHub sandbox, intake-esm (on S3 NetCDF)


Author: A.Radhakrishnan, Feb 2021

using t2.xlarge

In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import xarray as xr
import intake,yaml
import intake_esm
import numpy as np
%matplotlib inline

In [6]:
def launchDask(options):
    #TODO try, except
    gateway = Gateway()
    clusters = gateway.list_clusters()
    clusters
    if len(clusters) >0:
        cluster = gateway.connect(clusters[0].name)
        print("using existing connection")#or shut down and start afresh cluster.shutdown(). otherwise I dunno how to apply custom options
        cluster.shutdown()
        cluster = gateway.new_cluster(options)
    else:
        cluster = gateway.new_cluster(options)
    return(cluster)

In [7]:
gateway = Gateway()
options = gateway.cluster_options()

In [8]:
options.worker_memory=4

LAUNCH DASK CLUSTER 

In [9]:
options

In [10]:
cluster = launchDask(options)

In [11]:
cluster

In [12]:
min_w = 0
max_w = 60

In [13]:
from distributed import Client
client = Client(cluster)
client

Client Scheduler: gateway://traefik-daskhub-dask-gateway.daskhub:80/daskhub.c5adba3a70c048078eb9a9b05a9437bd Dashboard: /services/dask-gateway/clusters/daskhub.c5adba3a70c048078eb9a9b05a9437bd/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


MONITOR THE DASHBOARD from link above

In [14]:
options

ADAPTIVELY SCALE CLUSTER, so when nothing is going on we use minimum nodes in the setting below.

In [15]:
cluster.adapt(minimum=min_w, maximum=max_w)

We will now use our intake_esm catalog and the intake_esm API for data exploration and search.
Then use xarray and matplotlib as needed. 

Catalog: 

esgf-world.json is the ESM collections spec file for the netCDF data in the S3 bucket esgf-world. The cell below points to the most updated catalog (updated periodically using a CatalogBuilder API)

Other examples can be found here as well:

You can refer to https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/esm-collection-spec-examples for the most recent catalogs
More examples can be found in https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/examples 


In [16]:
col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json" 

In [17]:
col = intake.open_esm_datastore(col_url)

Examples to just search for what we want from the catalog

In [18]:
expname_filter = ['historical']
table_id_filter = 'Omon'
model_filter = 'GFDL-CM4'
variable_id_filter = ["thetao"]#,"volcello"]
ens_filter = "r1i1p1f1"
#version_filter = "v20190726"
grid_label="gn"
cat = col.search(grid_label="gn",experiment_id=expname_filter, mip_table=table_id_filter,variable=variable_id_filter,model=model_filter) #,version="v20190726")

In [19]:
dset_dict_t = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 1}},storage_options={'anon':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [20]:
ds_t= dset_dict_t['CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon']
ds_t.thetao

<xarray.DataArray 'thetao' (ensemble_member: 1, time: 1980, lev: 35, y: 1080, x: 1440)>
dask.array<broadcast_to, shape=(1, 1980, 35, 1080, 1440), dtype=float32, chunksize=(1, 1, 35, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x                (x) float64 -299.7 -299.5 -299.2 ... 59.53 59.78 60.03
  * y                (y) float64 -80.39 -80.31 -80.23 ... 89.73 89.84 89.95
  * lev              (lev) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    lon              (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat              (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
Attributes:
    long_name:      Sea Water Potential Temperature
    units:          degC
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello volume: volcello
    standard_name:  sea_water_potential_temperature
    original_name:  thetao

In [21]:
expname_filter = ['historical']
table_id_filter = 'Omon'
model_filter = 'GFDL-CM4'
variable_id_filter = ["volcello"]#,"volcello"]
ens_filter = "r1i1p1f1"
#version_filter = "v20190726"
grid_label="gn"
cat_v = col.search(grid_label="gn",experiment_id=expname_filter, mip_table=table_id_filter,variable=variable_id_filter,model=model_filter) #,version="v20190726")

In [22]:
dset_dict_v = cat_v.to_dataset_dict(cdf_kwargs={'chunks': {'time': 1}},storage_options={'anon':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [23]:
ds_v= dset_dict_v['CMIP6.NOAA-GFDL.GFDL-CM4.historical.Omon']
ds_v.volcello

<xarray.DataArray 'volcello' (ensemble_member: 1, time: 1980, lev: 35, y: 1080, x: 1440)>
dask.array<broadcast_to, shape=(1, 1980, 35, 1080, 1440), dtype=float32, chunksize=(1, 1, 35, 1080, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time             (time) object 1850-01-16 12:00:00 ... 2014-12-16 12:00:00
  * x                (x) float64 -299.7 -299.5 -299.2 ... 59.53 59.78 60.03
  * y                (y) float64 -80.39 -80.31 -80.23 ... 89.73 89.84 89.95
  * lev              (lev) float64 2.5 10.0 20.0 32.5 ... 5.5e+03 6e+03 6.5e+03
    lon              (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
    lat              (y, x) float32 dask.array<chunksize=(1080, 1440), meta=np.ndarray>
  * ensemble_member  (ensemble_member) <U8 'r1i1p1f1'
Attributes:
    long_name:      Ocean Grid-Cell Volume
    units:          m3
    cell_methods:   area: mean where sea time: mean
    cell_measures:  area: areacello volume: volcello
    standard_name:  ocean_volume
    original_name:  volcello

In [ ]:
#dset_dict = cat.to_dataset_dict(cdf_kwargs={'chunks': {'time': 1, 'lev': 35, 'y': 180, 'x': 360}},storage_options={'anon':True})
#CM4_T.chunk({'time': 1, 'lev': 35, 'lat': 180, 'lon': 360})

In [24]:
OHC = 3992 * 1025 * ds_t.thetao * ds_v.volcello

In [25]:
OHC

,Array,Chunk
Bytes,862.20 GB,435.46 MB
Shape,"(1, 1980, 35, 1080, 1440)","(1, 1, 35, 1080, 1440)"
Count,15858 Tasks,1980 Chunks
Type,float64,numpy.ndarray


In [26]:
global_OHC_upper700m = OHC.sel(lev=slice(0,700)).sum(dim=('lev', 'y', 'x'))

In [27]:
%time _ = global_OHC_upper700m.load()
#r5.xlarge max30 autoscaler, 4GB,1 core workers
#adaptive scaling 0,60; 
#workers used:60; nodes required: 21

#CPU times: user 8.82 s, sys: 427 ms, total: 9.25 s
#Wall time: 10min 39s

CPU times: user 8.82 s, sys: 427 ms, total: 9.25 s
Wall time: 10min 39s


In [ ]:
cluster.shutdown()